In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --use-deprecated=legacy-resolver git+https://github.com/pycaret/pycaret.git

In [ ]:
!pip install --use-deprecated=legacy-resolver pycaret[full]

In [ ]:
!pip install statsmodels
!pip install pystan~=2.14
!pip install fbprophet
!pip install jupyter-dash
!pip install ruptures

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fbprophet import Prophet
import ruptures as rpt
from sklearn.metrics import r2_score

## dash imports
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from pycaret.regression import *

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AQI/Data/2020-2022_history_data_processed.csv")
df.head()

,Station,City,State,Date,Time,PM25,PM10,CO,NO2,NH3,O3,SO2,AQI,AQI_bucket
0,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,10:00:00,32.76,37.19,0.57,6.28,11.40,130.18,12.88,176.0,Moderate
1,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,11:00:00,43.00,50.81,0.82,13.66,19.00,97.28,15.50,176.0,Moderate
2,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,12:00:00,51.04,61.88,1.04,22.77,25.08,63.66,17.17,176.0,Moderate
3,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,13:00:00,55.24,68.78,1.12,26.22,29.64,42.20,16.69,176.0,Moderate
4,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,14:00:00,58.45,73.32,1.12,24.04,29.89,33.98,15.26,176.0,Moderate


In [ ]:
data = df.groupby(['City','Date']).mean().reset_index()
data.head()

,City,Date,PM25,PM10,CO,NO2,NH3,O3,SO2,AQI
0,Agartala,2020-11-28,104.606429,124.784286,1.040000,14.719286,10.559286,43.337143,1.912857,245.642857
1,Agartala,2020-11-29,97.869167,115.230833,0.973750,11.354583,16.983750,62.903750,2.817083,252.041667
2,Agartala,2020-11-30,126.995417,147.259583,1.184583,17.886250,14.861667,60.678750,4.024167,274.833333
3,Agartala,2020-12-01,120.864167,140.599167,0.992917,11.260417,10.057083,67.114167,4.995000,294.750000
4,Agartala,2020-12-02,139.565833,164.991667,1.170417,12.983750,13.930417,58.895417,5.323750,312.708333


In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data['month'] = [i.month for i in data['Date']]
data['year'] = [i.year for i in data['Date']]
data['day_of_week'] = [i.dayofweek for i in data['Date']]
data['day_of_year'] = [i.dayofyear for i in data['Date']]

data.columns

Index(['City', 'Date', 'PM25', 'PM10', 'CO', 'NO2', 'NH3', 'O3', 'SO2', 'AQI',
       'month', 'year', 'day_of_week', 'day_of_year'],
      dtype='object')

In [ ]:
cities = data['City'].unique().tolist()

data=data.drop([ 'PM25', 'PM10', 'CO', 'NO2', 'NH3', 'O3', 'SO2'],axis=1)
data.set_index('Date',inplace=True)
data.head()

,City,AQI,month,year,day_of_week,day_of_year
Date,,,,,,
2020-11-28,Agartala,245.642857,11,2020,5,333
2020-11-29,Agartala,252.041667,11,2020,6,334
2020-11-30,Agartala,274.833333,11,2020,0,335
2020-12-01,Agartala,294.750000,12,2020,1,336
2020-12-02,Agartala,312.708333,12,2020,2,337


In [ ]:
dates = pd.date_range(start='2020-11-28', end = '2022-02-09', freq = 'D')
ma_df = pd.DataFrame()
ma_df['Date'] = dates
ma_df.set_index('Date',inplace=True)
ma_df.head()

""
Date
2020-11-28
2020-11-29
2020-11-30
2020-12-01
2020-12-02


In [ ]:
# plot multiple time series with moving avgs in a loop

for i in cities:
  subset = data[data['City'] == i]
  subset[f'{i}_MA'] = subset['AQI'].rolling(30).mean()
  subset.rename(columns = {'AQI': f'{i}_AQI'}, inplace = True)
  subset.drop(columns=['City','month','year','day_of_week','day_of_year'],inplace=True)
  ma_df = pd.merge(ma_df, subset, left_index=True, right_index=True)
ma_df.head()


,Agartala_AQI,Agartala_MA,Agra_AQI,Agra_MA,Ahmedabad_AQI,Ahmedabad_MA,Aizawl_AQI,Aizawl_MA,Ajmer_AQI,Ajmer_MA,Alwar_AQI,Alwar_MA,Amaravati_AQI,Amaravati_MA,Ambala_AQI,Ambala_MA,Amritsar_AQI,Amritsar_MA,Ankleshwar_AQI,Ankleshwar_MA,Araria_AQI,Araria_MA,Asansol_AQI,Asansol_MA,Aurangabad_AQI,Aurangabad_MA,Bagalkot_AQI,Bagalkot_MA,Bahadurgarh_AQI,Bahadurgarh_MA,Ballabgarh_AQI,Ballabgarh_MA,Bathinda_AQI,Bathinda_MA,Bengaluru_AQI,Bengaluru_MA,Bettiah_AQI,Bettiah_MA,Bhiwani_AQI,Bhiwani_MA,Bhopal_AQI,Bhopal_MA,Bihar Sharif_AQI,Bihar Sharif_MA,Bilaspur_AQI,Bilaspur_MA,Brajrajnagar_AQI,Brajrajnagar_MA,Bulandshahr_AQI,Bulandshahr_MA,Buxar_AQI,Buxar_MA,Chamarajanagar_AQI,Chamarajanagar_MA,Chandigarh_AQI,Chandigarh_MA,Chandrapur_AQI,Chandrapur_MA,Chennai_AQI,Chennai_MA,Chhapra_AQI,Chhapra_MA,Chikkaballapur_AQI,Chikkaballapur_MA,Chikkamagaluru_AQI,Chikkamagaluru_MA,Coimbatore_AQI,Coimbatore_MA,Damoh_AQI,Damoh_MA,Darbhanga_AQI,Darbhanga_MA,Davanagere_AQI,Davanagere_MA,Delhi_AQI,Delhi_MA,Dewas_AQI,Dewas_MA,Dharuhera_AQI,Dharuhera_MA,Durgapur_AQI,Durgapur_MA,Fatehabad_AQI,Fatehabad_MA,Firozabad_AQI,Firozabad_MA,Gadag_AQI,Gadag_MA,Gandhinagar_AQI,Gandhinagar_MA,Gaya_AQI,Gaya_MA,Gorakhpur_AQI,Gorakhpur_MA,Gummidipoondi_AQI,Gummidipoondi_MA,Gurgaon_AQI,Gurgaon_MA,Gurugram_AQI,Gurugram_MA,Guwahati_AQI,Guwahati_MA,Gwalior_AQI,Gwalior_MA,Haldia_AQI,Haldia_MA,Hapur_AQI,Hapur_MA,Hassan_AQI,Hassan_MA,Hisar_AQI,Hisar_MA,Howrah_AQI,Howrah_MA,Hubballi_AQI,Hubballi_MA,Hyderabad_AQI,Hyderabad_MA,Hyderabad US Consulate_AQI,Hyderabad US Consulate_MA,Indore_AQI,Indore_MA,Jabalpur_AQI,Jabalpur_MA,Jaipur_AQI,Jaipur_MA,Jalandhar_AQI,Jalandhar_MA,Jind_AQI,Jind_MA,Jodhpur_AQI,Jodhpur_MA,Jorapokhar_AQI,Jorapokhar_MA,Kaithal_AQI,Kaithal_MA,Kalaburagi_AQI,Kalaburagi_MA,Kalyan_AQI,Kalyan_MA,Kannur_AQI,Kannur_MA,Kanpur_AQI,Kanpur_MA,Karnal_AQI,Karnal_MA,Katihar_AQI,Katihar_MA,Katni_AQI,Katni_MA,Kishanganj_AQI,Kishanganj_MA,Kochi_AQI,Kochi_MA,Kohima_AQI,Kohima_MA,Kolar_AQI,Kolar_MA,Kollam_AQI,Kollam_MA,Koppal_AQI,Koppal_MA,Kota_AQI,Kota_MA,Kozhikode_AQI,Kozhikode_MA,Kurukshetra_AQI,Kurukshetra_MA,Lucknow_AQI,Lucknow_MA,Ludhiana_AQI,Ludhiana_MA,Madikeri_AQI,Madikeri_MA,Maihar_AQI,Maihar_MA,Mandi Gobindgarh_AQI,Mandi Gobindgarh_MA,Mandideep_AQI,Mandideep_MA,Mandikhera_AQI,Mandikhera_MA,Mangalore_AQI,Mangalore_MA,Meerut_AQI,Meerut_MA,Moradabad_AQI,Moradabad_MA,Motihari_AQI,Motihari_MA,Mumbai_AQI,Mumbai_MA,Munger_AQI,Munger_MA,Muzaffarnagar_AQI,Muzaffarnagar_MA,Muzaffarpur_AQI,Muzaffarpur_MA,Mysuru_AQI,Mysuru_MA,Nagpur_AQI,Nagpur_MA,Naharlagun_AQI,Naharlagun_MA,Nandesari_AQI,Nandesari_MA,Narnaul_AQI,Narnaul_MA,Nashik_AQI,Nashik_MA,Pali_AQI,Pali_MA,Patiala_AQI,Patiala_MA,Patna_AQI,Patna_MA,Pithampur_AQI,Pithampur_MA,Prayagraj_AQI,Prayagraj_MA,Puducherry_AQI,Puducherry_MA,Purnia_AQI,Purnia_MA,Raichur_AQI,Raichur_MA,Rajamahendravaram_AQI,Rajamahendravaram_MA,Rajgir_AQI,Rajgir_MA,Ramanagara_AQI,Ramanagara_MA,Ratlam_AQI,Ratlam_MA,Rohtak_AQI,Rohtak_MA,Rupnagar_AQI,Rupnagar_MA,Sagar_AQI,Sagar_MA,Saharsa_AQI,Saharsa_MA,Sasaram_AQI,Sasaram_MA,Satna_AQI,Satna_MA,Shivamogga_AQI,Shivamogga_MA,Siliguri_AQI,Siliguri_MA,Sirsa_AQI,Sirsa_MA,Solapur_AQI,Solapur_MA,Srinagar_AQI,Srinagar_MA,Talcher_AQI,Talcher_MA,Thiruvananthapuram_AQI,Thiruvananthapuram_MA,Thrissur_AQI,Thrissur_MA,Tirupati_AQI,Tirupati_MA,Udaipur_AQI,Udaipur_MA,Ujjain_AQI,Ujjain_MA,Vapi_AQI,Vapi_MA,Varanasi_AQI,Varanasi_MA,Vijayapura_AQI,Vijayapura_MA,Visakhapatnam_AQI,Visakhapatnam_MA,Vrindavan_AQI,Vrindavan_MA,Yadgir_AQI,Yadgir_MA
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-11-28,245.642857,NaN,251.000000,NaN,149.178571,NaN,45.571429,NaN,139.357143,NaN,149.285714,NaN,195.214286,NaN,316.071429,NaN,295.142857,NaN,180.642857,NaN,397.500000,NaN,308.428571,NaN,91.642857,NaN,58.571429,NaN,

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown",
        options=[{"label": x, "value": x} 
                 for x in cities],
        value="Delhi",
        placeholder="Select a city"
    ),
    dcc.Graph(id="line-chart")
])

@app.callback(
    Output("line-chart", "figure"), 
    Input("dropdown", "value"))
def update_line_chart(city):

    city_df = ma_df[[f'{city}_AQI',f'{city}_MA']]

    fig = px.line(data_frame=city_df,
        x=city_df.index, y=f'{city}_AQI', title = f'{city} Moving average and AQI', template = 'plotly_dark')  # 'plotly_dark'

    fig.add_trace(go.Scatter( x=city_df.index, y=city_df[f'{city}_MA'].to_numpy(), name="Moving Avg", line_shape='linear'))
    return fig

app.run_server(mode='inline')

<IPython.core.display.Javascript object>

### Model Training

In [ ]:
all_ts = data['City'].unique()
all_results = []
final_model = {}

for i in all_ts:
    df_subset = data[data['City'] == i]
    # initialize setup from pycaret.regression
    s = setup(df_subset, target = 'AQI', train_size = 0.95, transform_target = True, remove_outliers = True, data_split_shuffle = False,
              fold_strategy = 'timeseries', fold = 5, ignore_features = ['City'], numeric_features = ['day_of_year', 'year'],
              categorical_features = ['month', 'day_of_week'], silent = True, verbose = False, session_id = 2022)
    # compare all models and select best one based on MAE
    best_model = compare_models(sort = 'MAE', verbose=True)
    
    # capture the compare result grid and store best model in list
    p = pull().iloc[0:1]
    p['City'] = str(i)
    all_results.append(p)
    
    # finalize model i.e. fit on entire data including test set
    f = finalize_model(best_model)
    
    # attach final model to a dictionary
    final_model[i] = f
    
    # save transformation pipeline and model as pickle file 
    save_model(f, model_name='/content/drive/MyDrive/AQI/trained_models/' + str(i), verbose=False)

In [ ]:
concat_results = pd.concat(all_results,axis=0)

In [ ]:
concat_results.to_csv('/content/drive/MyDrive/AQI/best_models.csv',index=False)

In [ ]:
concat_results = pd.read_csv('/content/drive/MyDrive/AQI/best_models.csv')
concat_results.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),City
0,K Neighbors Regressor,65.5694,9608.1382,88.1737,-0.5412,0.6225,0.6994,0.078,Agartala
1,Random Forest Regressor,66.8475,6770.4807,81.4201,-0.4489,0.3940,0.3977,0.254,Agra
2,K Neighbors Regressor,49.7602,4519.7823,62.5954,-1.1665,0.4452,0.4434,0.084,Ahmedabad
3,Light Gradient Boosting Machine,31.3064,2493.7184,38.6279,-0.8173,0.5316,0.5958,0.176,Aizawl
4,Light Gradient Boosting Machine,37.1687,2927.5236,48.9237,-0.5802,0.3473,0.2737,0.174,Ajmer


### Forecasting

In [ ]:

all_dates = pd.date_range(start='2022-02-10', end = '2022-02-28', freq = 'D')
# create empty dataframe
score_df = pd.DataFrame()
# add columns to dataset
score_df['Date'] = all_dates
score_df['month'] = [i.month for i in score_df['Date']]
score_df['year'] = [i.year for i in score_df['Date']]
score_df['day_of_week'] = [i.dayofweek for i in score_df['Date']]
score_df['day_of_year'] = [i.dayofyear for i in score_df['Date']]
score_df.head()

,Date,month,year,day_of_week,day_of_year
0,2022-02-10,2,2022,3,41
1,2022-02-11,2,2022,4,42
2,2022-02-12,2,2022,5,43
3,2022-02-13,2,2022,6,44
4,2022-02-14,2,2022,0,45


In [ ]:
final_forecast = score_df.copy()
final_forecast.drop(columns=['month','year','day_of_week','day_of_year'],inplace=True)
final_forecast.head()

,Date
0,2022-02-10
1,2022-02-11
2,2022-02-12
3,2022-02-13
4,2022-02-14


In [ ]:
from pycaret.regression import load_model, predict_model
for i in cities:
    l = load_model('/content/drive/MyDrive/AQI/trained_models/' + str(i), verbose=False)
    p = predict_model(l, data=score_df)
    p.rename(columns = {'Label': f'{i}_Label'}, inplace = True)
    p.drop(columns=['month','year','day_of_week','day_of_year'],inplace=True)
    final_forecast = pd.merge(final_forecast, p, how = 'left', left_on='Date', right_on ='Date')

In [ ]:
final_forecast.head()

,Date,Agartala_Label,Agra_Label,Ahmedabad_Label,Aizawl_Label,Ajmer_Label,Alwar_Label,Amaravati_Label,Ambala_Label,Amritsar_Label,Ankleshwar_Label,Araria_Label,Asansol_Label,Aurangabad_Label,Bagalkot_Label,Bahadurgarh_Label,Ballabgarh_Label,Bathinda_Label,Bengaluru_Label,Bettiah_Label,Bhiwani_Label,Bhopal_Label,Bihar Sharif_Label,Bilaspur_Label,Brajrajnagar_Label,Bulandshahr_Label,Buxar_Label,Chamarajanagar_Label,Chandigarh_Label,Chandrapur_Label,Chennai_Label,Chhapra_Label,Chikkaballapur_Label,Chikkamagaluru_Label,Coimbatore_Label,Damoh_Label,Darbhanga_Label,Davanagere_Label,Delhi_Label,Dewas_Label,Dharuhera_Label,Durgapur_Label,Fatehabad_Label,Firozabad_Label,Gadag_Label,Gandhinagar_Label,Gaya_Label,Gorakhpur_Label,Gummidipoondi_Label,Gurgaon_Label,Gurugram_Label,Guwahati_Label,Gwalior_Label,Haldia_Label,Hapur_Label,Hassan_Label,Hisar_Label,Howrah_Label,Hubballi_Label,Hyderabad_Label,Hyderabad US Consulate_Label,Indore_Label,Jabalpur_Label,Jaipur_Label,Jalandhar_Label,Jind_Label,Jodhpur_Label,Jorapokhar_Label,Kaithal_Label,Kalaburagi_Label,Kalyan_Label,Kannur_Label,Kanpur_Label,Karnal_Label,Katihar_Label,Katni_Label,Kishanganj_Label,Kochi_Label,Kohima_Label,Kolar_Label,Kollam_Label,Koppal_Label,Kota_Label,Kozhikode_Label,Kurukshetra_Label,Lucknow_Label,Ludhiana_Label,Madikeri_Label,Maihar_Label,Mandi Gobindgarh_Label,Mandideep_Label,Mandikhera_Label,Mangalore_Label,Meerut_Label,Moradabad_Label,Motihari_Label,Mumbai_Label,Munger_Label,Muzaffarnagar_Label,Muzaffarpur_Label,Mysuru_Label,Nagpur_Label,Naharlagun_Label,Nandesari_Label,Narnaul_Label,Nashik_Label,Pali_Label,Patiala_Label,Patna_Label,Pithampur_Label,Prayagraj_Label,Puducherry_Label,Purnia_Label,Raichur_Label,Rajamahendravaram_Label,Rajgir_Label,Ramanagara_Label,Ratlam_Label,Rohtak_Label,Rupnagar_Label,Sagar_Label,Saharsa_Label,Sasaram_Label,Satna_Label,Shivamogga_Label,Siliguri_Label,Sirsa_Label,Solapur_Label,Srinagar_Label,Talcher_Label,Thiruvananthapuram_Label,Thrissur_Label,Tirupati_Label,Udaipur_Label,Ujjain_Label,Vapi_Label,Varanasi_Label,Vijayapura_Label,Visakhapatnam_Label,Vrindavan_Label,Yadgir_Label
0,2022-02-10,282.849583,392.174870,223.255783,109.702691,141.554414,232.628068,138.234585,373.230794,440.840105,183.699411,375.678390,316.569300,148.091662,182.953136,597.978859,533.824533,319.626188,136.649490,416.502396,382.621521,133.159485,397.573174,182.391731,224.646823,446.320754,400.310105,110.766436,313.726924,176.395467,142.073238,409.493711,124.349673,100.322182,146.154577,116.178940,440.242083,139.568448,593.331800,148.446594,356.163579,325.957642,328.456795,343.126804,172.428403,179.543455,341.693413,414.158884,103.010796,468.759916,619.178076,175.254449,226.711959,408.443189,446.322695,143.424461,312.694478,649.764089,162.458410,174.038670,277.156641,150.766301,139.256812,262.040488,418.911011,377.454635,150.743454,312.481722,450.453188,179.868756,215.631485,127.920927,410.492512,399.072456,400.853426,128.518829,356.533170,157.277519,96.466717,125.142069,140.167709,178.017001,243.497134,216.912906,392.087438,410.423593,413.449885,124.878410,141.962247,385.023608,163.907162,361.702583,132.248681,448.713864,426.387974,435.305339,297.110973,408.088574,418.323491,424.305227,143.569500,252.455055,51.984915,225.610114,245.017023,157.243927,123.370742,394.130591,419.056225,130.714238,391.726897,109.723976,401.920641,149.888840,146.052628,365.017160,134.987317,111.830205,343.634666,278.786926,100.833336,424.717167,331.886411,150.741123,120.842102,197.725803,298.935857,191.848764,304.607147,271.237972,137.262224,146.398113,111.365949,122.308792,127.757088,165.304027,417.942287,197.000008,326.009776,364.396521,163.284207
1,2022-02-11,298.196157,431.406002,254.420219,108.095165,153.968265,274.842821,166.495448,391.860669,511.381264,167.672332,374.341936,318.955894,147.849103,173.435384,578.203234,584.416361,351.516662,137.711411,421.904052,351.967194,125.127441,413.471508,193.089054,243.767836,443.115248,387.366296,110.757357,336.940722,173.876153,149.519497,43

In [ ]:
final_forecast.to_csv("/content/drive/MyDrive/AQI/final_forecast.csv",index=False)

In [ ]:
final_forecast = pd.read_csv('/content/drive/MyDrive/AQI/final_forecast.csv')
final_forecast.head()

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown",
        options=[{"label": x, "value": x} 
                 for x in cities],
        value="Delhi",
        placeholder="Select a city"
    ),
    dcc.Graph(id="line-chart")
])

@app.callback(
    Output("line-chart", "figure"), 
    Input("dropdown", "value"))
def update_line_chart(city):

    city_df = ma_df[[f'{city}_AQI']]
    fig = px.line(data_frame=city_df,
        x=city_df.index, y=f'{city}_AQI', title = f'{city} AQI Forcasting', template = 'plotly_dark') 

    fig.add_trace(go.Scatter( x=final_forecast.Date, y=final_forecast[f'{city}_Label'].to_numpy(), name="Forecast", line_shape='linear'))
    return fig

app.run_server(mode='inline')

<IPython.core.display.Javascript object>

Delhi, Mumbai, bangluru, hyderabad, chennai

In [ ]:
final_forecast.head()

,Date,Agartala_Label,Agra_Label,Ahmedabad_Label,Aizawl_Label,Ajmer_Label,Alwar_Label,Amaravati_Label,Ambala_Label,Amritsar_Label,Ankleshwar_Label,Araria_Label,Asansol_Label,Aurangabad_Label,Bagalkot_Label,Bahadurgarh_Label,Ballabgarh_Label,Bathinda_Label,Bengaluru_Label,Bettiah_Label,Bhiwani_Label,Bhopal_Label,Bihar Sharif_Label,Bilaspur_Label,Brajrajnagar_Label,Bulandshahr_Label,Buxar_Label,Chamarajanagar_Label,Chandigarh_Label,Chandrapur_Label,Chennai_Label,Chhapra_Label,Chikkaballapur_Label,Chikkamagaluru_Label,Coimbatore_Label,Damoh_Label,Darbhanga_Label,Davanagere_Label,Delhi_Label,Dewas_Label,Dharuhera_Label,Durgapur_Label,Fatehabad_Label,Firozabad_Label,Gadag_Label,Gandhinagar_Label,Gaya_Label,Gorakhpur_Label,Gummidipoondi_Label,Gurgaon_Label,Gurugram_Label,Guwahati_Label,Gwalior_Label,Haldia_Label,Hapur_Label,Hassan_Label,Hisar_Label,Howrah_Label,Hubballi_Label,Hyderabad_Label,Hyderabad US Consulate_Label,Indore_Label,Jabalpur_Label,Jaipur_Label,Jalandhar_Label,Jind_Label,Jodhpur_Label,Jorapokhar_Label,Kaithal_Label,Kalaburagi_Label,Kalyan_Label,Kannur_Label,Kanpur_Label,Karnal_Label,Katihar_Label,Katni_Label,Kishanganj_Label,Kochi_Label,Kohima_Label,Kolar_Label,Kollam_Label,Koppal_Label,Kota_Label,Kozhikode_Label,Kurukshetra_Label,Lucknow_Label,Ludhiana_Label,Madikeri_Label,Maihar_Label,Mandi Gobindgarh_Label,Mandideep_Label,Mandikhera_Label,Mangalore_Label,Meerut_Label,Moradabad_Label,Motihari_Label,Mumbai_Label,Munger_Label,Muzaffarnagar_Label,Muzaffarpur_Label,Mysuru_Label,Nagpur_Label,Naharlagun_Label,Nandesari_Label,Narnaul_Label,Nashik_Label,Pali_Label,Patiala_Label,Patna_Label,Pithampur_Label,Prayagraj_Label,Puducherry_Label,Purnia_Label,Raichur_Label,Rajamahendravaram_Label,Rajgir_Label,Ramanagara_Label,Ratlam_Label,Rohtak_Label,Rupnagar_Label,Sagar_Label,Saharsa_Label,Sasaram_Label,Satna_Label,Shivamogga_Label,Siliguri_Label,Sirsa_Label,Solapur_Label,Srinagar_Label,Talcher_Label,Thiruvananthapuram_Label,Thrissur_Label,Tirupati_Label,Udaipur_Label,Ujjain_Label,Vapi_Label,Varanasi_Label,Vijayapura_Label,Visakhapatnam_Label,Vrindavan_Label,Yadgir_Label
0,2022-02-10,282.849583,392.174870,223.255783,109.702691,141.554414,232.628068,138.234585,373.230794,440.840105,183.699411,375.678390,316.569300,148.091662,182.953136,597.978859,533.824533,319.626188,136.649490,416.502396,382.62152,133.15948,397.573174,182.391731,224.646823,446.320754,400.310105,110.766436,313.726924,176.395467,142.073238,409.493711,124.349673,100.322182,146.154577,116.17894,440.242083,139.568448,593.331800,148.446594,356.163579,325.957642,328.456795,343.126804,172.428403,179.543455,341.693413,414.158884,103.010796,468.759916,619.178076,175.254449,226.711959,408.443189,446.322695,143.424461,312.694478,649.764089,162.458410,174.038670,277.156641,150.766301,139.256812,262.040488,418.911011,377.454635,150.74345,312.481722,450.453188,179.868756,215.631485,127.920927,410.492512,399.072456,400.853426,128.51883,356.533170,157.277519,96.466717,125.142069,140.167709,178.017001,243.497134,216.912906,392.087438,410.423593,413.449885,124.878410,141.962247,385.023608,163.907162,361.702583,132.248681,448.713864,426.387974,435.305339,297.110973,408.088574,418.323491,424.305227,143.569500,252.455055,51.984915,225.610114,245.017023,157.243927,123.370742,394.130591,419.056225,130.714238,391.726897,109.723976,401.920641,149.888840,146.052628,365.017160,134.987317,111.830205,343.634666,278.786926,100.833336,424.717167,331.886411,150.741123,120.842102,197.725803,298.935857,191.848764,304.60715,271.237972,137.262224,146.398113,111.365949,122.308792,127.757088,165.304027,417.942287,197.000008,326.009776,364.396521,163.284207
1,2022-02-11,298.196157,431.406002,254.420219,108.095165,153.968265,274.842821,166.495448,391.860669,511.381264,167.672332,374.341936,318.955894,147.849103,173.435384,578.203234,584.416361,351.516662,137.711410,421.904052,351.96720,125.12744,413.471508,193.089054,243.767836,443.115248,387.366296,110.757357,336.940722,173.876153,149.519497,431.122764

In [ ]:
new_df = pd.DataFrame(columns=['Date','City','AQI_forecasted'])
new_df.head()

,Date,City,AQI_forecasted


In [ ]:
for i in cities:
  subset = final_forecast[['Date',f'{i}_Label']]
  subset['City'] = i
  subset.rename(columns = {f'{i}_Label': "AQI_forecasted"}, inplace = True)
  new_df = new_df.append(subset, ignore_index=True)

new_df.head()

,Date,City,AQI_forecasted
0,2022-02-10,Agartala,282.849583
1,2022-02-11,Agartala,298.196157
2,2022-02-12,Agartala,332.671042
3,2022-02-13,Agartala,355.694277
4,2022-02-14,Agartala,365.378408


In [ ]:
forecast_df = new_df.sort_values(['Date','City']).reset_index(drop=True)

In [ ]:
forecast_df.head()

,Date,City,AQI_forecasted
0,2022-02-10,Agartala,282.849583
1,2022-02-10,Agra,392.174870
2,2022-02-10,Ahmedabad,223.255783
3,2022-02-10,Aizawl,109.702691
4,2022-02-10,Ajmer,141.554414


In [ ]:
forecast_df.to_csv('/content/drive/MyDrive/AQI/forecast_aqi.csv',index=False)